In [72]:
import os
import os.path as osp
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import json
from itertools import chain
import copy

import Bio
from Bio import SeqIO
from Bio import AlignIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Align import substitution_matrices

import matplotlib.pyplot as plt

In [53]:
def load_lib(jsonlf):
    data = {
        'accession': [],
        'length': [],
        'location': [],
        'releaseDate': [],
    }
    for line in tqdm(jsonlf, position=0, leave=True, desc='reading lib file'):
        info = json.loads(line)
        if info['completeness']=='COMPLETE' and info['length']>27000:
            try:
                data['location'].append(info['location']['geographicLocation'])
            except:
                data['location'].append('Unknown')
            data['accession'].append(info['accession'])
            data['length'].append(info['length'])
            data['releaseDate'].append(info['releaseDate'])
    libdf = pd.DataFrame(data=data)
    libdf = libdf.sort_values(by=['releaseDate']).dropna()
    
    return libdf


def load_seq(fastapath, mode):
    sequences = {}
    for seq_record in tqdm(SeqIO.parse(fastapath,mode), position=0, leave=True, desc='reading fasta file'):
        sequences[seq_record.id] = seq_record.seq.replace('N','')
    
    return sequences

In [3]:
all_genomic_path = './../ncbi_dataset/data/genomic.fna'
complete_genomic_path = './../ncbi_dataset/data/complete_genome.fasta'
protein_path = './../ncbi_dataset/data/protein.faa'
libpath = './../ncbi_dataset/data/data_report.jsonl'

In [32]:
%%time
jsonlf = open(libpath,'r').readlines()
lib = load_lib(jsonlf)

reading lib file: 100%|██████████| 4542920/4542920 [10:15<00:00, 7379.33it/s]


CPU times: user 10min 27s, sys: 1min 41s, total: 12min 9s
Wall time: 15min 35s


In [54]:
%%time
sequences = load_seq(complete_genomic_path, 'fasta')

reading fasta file: 1001653it [03:19, 5014.38it/s] 


CPU times: user 1min 33s, sys: 26.9 s, total: 2min
Wall time: 3min 20s


In [55]:
acclist = lib.accession.tolist()
maxlen = lib.length.max()
minlen = lib.length.min()

In [56]:
print(len(acclist))
print(minlen, maxlen)

1001653
27366 30119


In [57]:
liblen = lib.sort_values(by=['length'],ascending=False).dropna()

In [58]:
print(liblen.head())

         accession  length                         location releaseDate
70312   MT844089.1   30119                          Unknown  2020-09-07
879886  MT079851.1   30018                            China  2020-05-15
385503  MT079845.1   29955                            China  2020-05-15
17088   MT121215.1   29945                  China: Shanghai  2020-03-09
614766  MZ472103.1   29940  USA: Kentucky, Jefferson County  2021-08-13


In [39]:
seq_stat = np.zeros((6,maxlen)) # A T C G N -
maxlen_seq = str(sequences[lib[lib.length==maxlen].accession.item()])
for j in range(len(maxlen_seq)):
    if maxlen_seq[j] == 'A': seq_stat[0,j] += 1
    elif maxlen_seq[j] == 'T': seq_stat[1,j] += 1
    elif maxlen_seq[j] == 'C': seq_stat[2,j] += 1
    elif maxlen_seq[j] == 'G': seq_stat[3,j] += 1
    elif maxlen_seq[j] == 'N': seq_stat[4,j] += 1
    elif maxlen_seq[j] == '-': seq_stat[5,j] += 1

In [ ]:
for i in range(1,len(acclist)):
    seq2 = sequences[acclist[i]]
    alignment = pairwise2.align.globalxx(seq1, seq2, penalize_end_gaps=False)

In [ ]:
for i in range(len(acclist)-1):
    seq1, seq2 = sequences[acclist[i]], sequences[acclist[i+1]]
    alignment = pairwise2.align.globalxx(seq1, seq2, penalize_end_gaps=False, one_alignment_only=True)
    seqB = str(alignment[0].seqB)
    for k in range(len(seqB)):
        if seqB[k] == 'A': seq_stat[0,k] += 1
        elif seqB[k] == 'T': seq_stat[1,k] += 1
        elif seqB[k] == 'C': seq_stat[2,k] += 1
        elif seqB[k] == 'G': seq_stat[3,k] += 1
        elif seqB[k] == 'N': seq_stat[4,k] += 1
        elif seqB[k] == '-': seq_stat[5,k] += 1

[[2. 0. 0. ... 1. 1. 1.]
 [0. 2. 2. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
[[3. 0. 0. ... 1. 1. 1.]
 [0. 3. 3. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 


In [59]:
minlen_seq = str(sequences[lib[lib.length==minlen].accession.item()])

In [76]:
%%time
alignments = pairwise2.align.globalxx(maxlen_seq, minlen_seq)

CPU times: user 1min 20s, sys: 26.4 s, total: 1min 47s
Wall time: 1min 47s


In [77]:
print(len(alignments))

19


In [82]:
for a in alignments:
    print(len(a.seqA))
    print(len(a.seqB))
    print()

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180

30180
30180



In [33]:
acclist = lib['accession'].tolist()
outfasta = './../ncbi_dataset/data/complete_genome.fasta'
with open(outfasta,'w') as f:
    for acc in tqdm(acclist, position=0, leave=True, desc='writing new fasta file'):
        content = '>'+acc+'\n'+str(sequences[acc])+'\n'
        f.write(content)

writing new fasta file: 100%|██████████| 1001653/1001653 [01:09<00:00, 14407.87it/s]
